In [18]:
# load necessary modules
import numpy as np 
from scipy.integrate import odeint
import os, sys 
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import utility as ut
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import time
import KS

save_folder='../data/KS-trajectories'

@ut.timer
def generate_data(i, N_data):
    # define data and initialize simulation
    L    = 200 / 2*np.pi
    N    = 512
    dt   = 0.25 / (i+1)
    ninittransients = 40000
    tend = (N_data + ninittransients) 
    dns  = KS.KS(L=L, N=N, dt=dt, nsteps=None, tend=tend)
    dl_max = 20000
    pl_max = 20000
    # simulate initial transient
    dns.simulate(nsteps=(N_data + ninittransients))
    # convert to physical space
    dns.fou2real()
    return dns.uu[ninittransients:]

u = generate_data(0, int(1e5))
u.shape

Time taken by simulate is 0.0142 seconds
Time taken by generate_data is 0.0268 seconds


(0, 512)

In [11]:
# generate test data
u_test = generate_data(0, int(500*1000))
test = u_test[1:].reshape(500,-1, 512)
test = np.moveaxis(test, -1, -2)
np.random.shuffle(test)
np.save('{}/{}.npy'.format(save_folder, f'test'), test)
test.shape

Time taken by simulate is 0.0162 seconds
Time taken by generate_data is 0.0233 seconds


(500, 512, 0)

In [5]:
# generate training data
for i in [0]:
    u = generate_data(i, int(2e5)).T
    np.save('{}/{}.npy'.format(save_folder, f'train{i+1}'), u)

Time taken by simulate is 0.0157 seconds
Time taken by generate_data is 0.0257 seconds


In [8]:
i = 0
u = np.load('{}/{}.npy'.format(save_folder, f'train{i+1}'))

In [3]:
u.dtype

dtype('float64')

In [15]:
np.save('{}/{}.npy'.format(save_folder, f'test'), test)

In [17]:
np.random.shuffle(test)

In [18]:
np.save('{}/{}.npy'.format(save_folder, f'test'), test[:500, :, :])

In [6]:
u

array([], shape=(512, 0), dtype=float64)

In [12]:
np.real(np.fft.ifft([0, 4, 0, 0])).dtype

dtype('float64')